# API access of the OpenTopography catalog

## Authors
Cassandra Brigham<sup>1,2</sup>

<sup>1</sup>Arizona State University <sup>2</sup>OpenTopography

## Table of Contents

* [1. Purpose](#1-purpose)  
* [2. Setup](#2-setup)  
    * [2.1. API Key](#21-api-key)  
    * [2.2. Installation Options](#22-installation-options)  
        * [2.2.1. Option 1: Install and run on Google Colaboratory](#221-option-1-install-and-run-on-google-colaboratory)  
        * [2.2.2. Option 2: Download this notebook and run locally](#222-option-2-download-this-notebook-and-run-locally)  
    * [2.3. Library Imports](#23-library-imports)  
    * [2.4. Define Functions](#24-define-functions)  
* [3. Data Access and Processing](#3-data-access-and-processing)  
    * [3.1. Define Area of Interest](#31-define-area-of-interest)  
        * [3.1.1. Option 1: Draw a bounding box on an interactive map](#311-option-1-draw-a-bounding-box-on-an-interactive-map)  
        * [3.1.2. Option 2: Define bounds manually](#312-option-2-define-bounds-manually)  
        * [3.1.3. Option 3: Define bounds using an uploaded shapefile](#313-option-3-define-bounds-using-an-uploaded-shapefile)  
    * [3.2. Use OT catalog to find datasets](#32-use-ot-catalog-to-find-datasets)  
* [4. Conclusions](#4-conclusions)  
* [5. Resources](#5-resources)  
* [6. Funding, Keywords, and Citation](#6-funding-keywords-and-citation)

## 1. Purpose

OpenTopography (OT) is a National Science Foundation–funded cyberinfrastructure facility hosted at the San Diego Supercomputer Center (SDSC) at UC San Diego in partnership with the EarthScope Consortium and Arizona State University. Its mission is to democratize online access to high-resolution topographic and bathymetric data acquired via lidar, radar, and photogrammetry, and to provide value‑added tools for data discovery, access, processing, and visualization in the cloud (opentopography.org).

As a distributed facility, OpenTopography aggregates and federates datasets from multiple providers — including USGS 3DEP, NOAA coastal lidar, and Polar Geospatial Center’s ArcticDEM and REMA products — into a unified catalog. Data are accessible under open licenses, with professional training and guidance offered for effective use of the platform.

To support programmatic workflows, OpenTopography offers <a href = "https://portal.opentopography.org/apidocs/"> a suite of RESTful web services documented in OpenAPI</a>, including:

* <a href = ""> Global Datasets API </a> for global DEMs (SRTM, ALOS, NASADEM, Copernicus DSM, etc.)

* <a href = ""> USGS 3DEP Raster API </a> for accessing 1 m, 10 m, and 30 m USGS DEM products

* <a href = ""> <strong> Data Catalog API </strong> </a> (this notebook’s focus) for bounding‑box search of hosted and federated point cloud and raster datasets

This and the other use-case specific Jupyter Notebooks developed as part of this effort for API use are available in a <a href=""> Github repository </a> and may be run locally or on the <a href="">Google Colaboratory</a> cloud platform.

API access requires a free OT API key for academic users (rate limited to 500 calls/24 h for academic users) and supports JSON or XML output formats. Key access and management is available via the MyOpenTopo portal. For users who wish to integrate the OT API into commercial software or who need higher processing limits, information about Enterprise keys can be found <a href="https://opentopography.org/about/partner">here</a>.

By leveraging OpenTopography’s scalable, web‑service infrastructure, this notebook streamlines dataset discovery workflows and lays the groundwork for integrating OT data into PDAL/GDAL‑based geospatial pipelines.

#### Specific features of this notebook

1. Interactive AOI definition: Choose between manual coordinate entry, uploading a shapefile, or drawing a bounding box directly on an embedded ipyleaflet map.

2. Automated OT Catalog queries: Build parameterized requests to the /otCatalog API endpoint, including WGS84 bounding boxes or WKT polygons, to fetch dataset metadata in JSON format. 

3. Result export: Save raw API responses to disk (results.json) for reproducibility and offline inspection.

4. Metadata parsing and display: Extract key attributes (e.g., dataset name, bounds, acquisition date) from API results and present them in a clear, tabular summary.

5. Modular code structure: Encapsulate query logic in functions to enable easy reuse and integration into larger PDAL/GDAL-based processing workflows.


## 2. Setup

### 2.1. API Key
We recommend storing your OT API key in a environment variable. This prevents keys from being hardcoded in the source code, reducing the risk of exposure through sharing or version control. It also enhances flexibility, allowing the same code to be used in various environments without changes. Below are the steps to take to store your API key in an environment variable.  

#### For Linux/macOS  

Open up a Terminal window and find your shell's profile script. For Bash, you might find '~ /.bashrc' or '~ /.bash_profile'.
For Zsh, you might find '~ /.zshrc'.  

```cd ~```  
```ls -a```  

Once you know the name of your shell’s profile script, you can edit it using a text editor that operates in the terminal (like ```nano``` or ```vim```) or out side the terminal with the text editor of your choosing. This example will use '~/.zshrc' as the name of the shell profile script and ```nano``` as the text editor. If the file is read-only, you might need to use ```sudo``` to edit it.  

```nano ~/.zshrc```  

At the end of the .zshrc file, add a line to define your environment variable. 'your_api_key_here' is a stand in for the alphanumeric API key accessible at [MyOpenTopo](https://portal.opentopography.org/myopentopo) under "Get an API Key."   
 
```export OPENTOPOGRAPHY_API_KEY='your_api_key_here'```


Exit (```control + X``` for ```nano```) and save (```Y``` to ```Save modified buffer (ANSWERING "No" WILL DESTROY CHANGES) ?``` then ```Enter``` for ```nano```).

For your changes to take effect, you need to reload the .zshrc file or restart your terminal. To reload .zshrc without restarting, type the following command in your terminal and press Enter:  

```source ~/.zshrc```  

This will make the OPENTOPOGRAPHY_API_KEY environment variable available in all new terminal sessions.  

#### For Windows  

1. Search for "Environment Variables" in the Start menu.
1. Click on "Edit the system environment variables."
1. In the System Properties window, click on "Environment Variables."
1. Click on "New" under System variables or User variables depending on your need.
1. Set "Variable name" as OPENTOPOGRAPHY_API_KEY and "Variable value" as your actual API key. Your alphanumeric API key is accessible at [MyOpenTopo](https://portal.opentopography.org/myopentopo) under "Get an API Key."
1. Click OK and apply the changes.

### 2.2. Installation Options
There are two options for performing the workflow steps outlined below. **Option 1** is our suggested method for simplicity, as building a virtual environment with the required dependencies on the user's local file system can be challenging based on the user's experience level with Python and <a href="https://www.anaconda.com/"> Anaconda</a>.

1. **Option 1**: Launch the interactive Jupyter notebook on Google Colaboratory.
    - Does not require creation of a virtual environment or installation on local file system.
    - Requires Google account and access to personal Google Drive folder.
    - Data download limits will be dependent on user's available Google Drive storage. 
    - If you wish to run this notebook in Google Colaboratory click the 'Open in Colab' badge below. 
    <br/><br/>
2. **Option 2**: Download this Jupyter notebook (.ipynb file) to your local file system.
    - Create a virtual conda environment containing the required dependencies.
    - Run Juypter notebook on local machine.
    - Data download limits and computation speed will be dependent on user's hardware.

### 2.2.1. Option 1: Install and run on Google Colaboratory
For ease-of-use, it is suggested to launch and execute these notebooks on <a href="https://colab.research.google.com/">Google Colaboratory</a> (Colab, for short), Google's Cloud Platform. Dependencies will be installed on a virtual machine on Google's cloud servers and the code will be executed directly in your browser! A major benefit of this is that you will have direct access to Google's high-end CPU/GPUs and will not have to install any dependencies locally. All deliverables will be saved to your personal Google Drive. To experiment and run one of the below Jupyter Notebooks on Google Colab click the "Open in Colab" badge below.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cmspeed/OT_3DEP_Workflows/blob/main/notebooks/01_3DEP_Generate_DEM_User_AOI.ipynb)

In [ ]:
# This cell only excecutes if you're running on Colab. Installation process takes 2-3 minutes.
import os, sys
if 'google.colab' in sys.modules:
    
  # Mount Google Drive. You will be prompted to grant file I/O access to Drive.
  from google.colab import drive 
  drive.mount('/gdrive/') # Mount Google Drive! 

  # Clone OpenTopography 3DEP Workflow Git Repository
  !git clone https://github.com/Cassandra-Brigham/OT_API_notebooks

  #  Install the core dependencies (other than PDAL/GDAL) from requirements.txt
  !pip install -r OT_API_notebooks/requirements.txt

  # Install Conda (necessary to install PDAL/GDAL)
  !pip install -q condacolab
  import condacolab
  condacolab.install()

  #kernel will restart. Install PDAL and GDAL with Mamba.
  !mamba install -q python-pdal gdal
  
  # Runtime will restart automatically. Do not rerun above cells.

In [ ]:
# This cell only excecutes if you're running on Colab.
import os, sys
if 'google.colab' in sys.modules:
    # Colab requires proj_lib environment variable to be set manually.
    os.environ['PROJ_LIB'] = '/usr/local/share/proj/'

    !pip install python-dotenv  # if not already installed

    from dotenv import set_key, find_dotenv

    # 1) Locate (or create) your .env
    #    find_dotenv returns the first .env in cwd hierarchy, or '' if none
    env_path = find_dotenv(usecwd=True)
    if not env_path:
        env_path = ".env"
        open(env_path, "a").close()

    # 2) Define your variables
    variables = {
        "OPENTOPOGRAPHY_API_KEY": os.getenv("OPENTOPOGRAPHY_API_KEY", "default_value"),
    }

    # 3) Write/update each key in the .env
    for key, val in variables.items():
        set_key(env_path, key, val)

    print(f".env updated at {env_path}")

**If using Option 1 (Google Colab), proceed to Library Imports**

<a id="222-option-2-download-this-notebook-and-run-locally"></a>
### 2.2.2. Option 2: Install and run on local file system

If you would like to run the Jupyter Notebook on your local machine:

Make a new directory on your local file system where the 3DEP Jupyter Notebooks (and all 3DEP data, if desired) will be saved. In this example case, the directory will be called `3DEP`.

```bash  
    mkdir OT_API
```

Change into the new directory and `git clone` the Github repository containing the Jupyter Notebooks and other relevant files to your local file system.

```bash 
    cd OT_API
    git clone https://github.com/
```


You will need to configure your Python environment using your preferred environment management system (e.g., Conda, virtualenv, pipenv). If you don’t already use one, we provide a helper script that:

1. Detects your operating system  
2. Installs Miniconda automatically on macOS or Linux(Windows users will need to install Miniconda manually following the instructions at https://docs.conda.io/en/latest/miniconda.html)

First, make the installer executable and run it in your terminal:

```bash
    chmod +x install-miniconda.sh
    ./install-miniconda.sh
```

Once Miniconda is installed (or if you already have Conda), create the environment from the provided `environment.yml`:

```bash
    conda env create -f environment.yml
```

After the environment is created, activate it and launch Jupyter Notebook:

```bash
    conda activate ot_env
```

Now, launch the chosen Jupyter Notebook. If unsure how to launch a Notebook, refer to this guide (https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/execute.html).

**You may now proceed to Library Imports**

<a id="#23-library-imports"></a>
### 2.3. Library Imports

After successfully completing the steps outlined in either **Option 1** or **Option 2**, we can now import the modules for use throughout the rest of the notebook.

In [55]:
import requests
from ipyleaflet import Map, DrawControl, basemaps, GeoJSON, LegendControl
import matplotlib.pyplot as plt
from shapely import wkt
from shapely.geometry import box, shape
import geopandas as gpd

<a id="#24-define-functions"></a>
### 2.4. Define Functions

Several helper functions are provided in the cell below. These functions are necessary for successful execution of the remainder of the notebook. Broadly, they provide the utilities to:

- draw an area of interest (AOI) on an interactive `ipyleaflet` map  
- fetch and layer OpenTopography, USGS 3DEP, and NOAA boundary GeoJSON from GitHub  
- capture and expose AOI extents (south, north, west, east) and a WKT polygon for downstream API queries  

The primary function, `init_ot_catalog_map()`, encapsulates map initialization, draw‐control setup, boundary‐layer loading, and legend creation, and returns both the map object and a `bounds` dict with keys  
```python
{
  'south': …,   # minimum latitude
  'north': …,   # maximum latitude
  'west':  …,   # minimum longitude
  'east':  …,   # maximum longitude
  'polygon': …  # WKT representation of the drawn rectangle
}
```

**These functions can be modified as the user sees fit; however, they are designed to work with a simple execution of the below cell.**

In [ ]:
# 
def geojson_to_wkt(geojson):
    # Ensure the input is a Polygon
    if geojson['type'] != 'Polygon':
        raise ValueError("Input must be a Polygon")

    # Extract coordinates
    coordinates = geojson['coordinates']
    
    # Convert coordinates to WKT string
    wkt_coordinates = ', '.join(
        f"{', '.join(f'{x}, {y}' for x, y in polygon)}"
        for polygon in coordinates
    )
    
    return wkt_coordinates

def init_ot_catalog_map(
    center=(39.8283, -98.5795),
    zoom=3,
    three_dep_url="https://raw.githubusercontent.com/OpenTopography/Data_Catalog_Spatial_Boundaries/main/usgs_3dep_boundaries.geojson",
    noaa_url="https://raw.githubusercontent.com/OpenTopography/Data_Catalog_Spatial_Boundaries/main/noaa_coastal_lidar_boundaries.geojson",
    ot_url="https://raw.githubusercontent.com/OpenTopography/Data_Catalog_Spatial_Boundaries/main/OT_PC_boundaries.geojson"
):
    """
    Initialize an ipyleaflet Map with:
      - a DrawControl to capture an AOI bounding box (WKT + coords)
      - 3DEP, NOAA, and OT dataset boundary layers
      - a legend
    
    Returns:
      m      -- the Map object
      bounds -- dict with keys 'south', 'north', 'west', 'east', 'polygon'
    """
    # storage for bounds
    bounds = {'south': None, 'north': None, 'west': None, 'east': None, 'polygon': None}

    # drawing callback
    def _handle_draw(self, action, geo_json):
        coords = geo_json['geometry']['coordinates'][0]
        bounds['south'] = coords[0][1]
        bounds['west']  = coords[0][0]
        bounds['north'] = coords[2][1]
        bounds['east']  = coords[2][0]
        # convert GeoJSON to WKT
        geom = shape(geo_json['geometry'])
        bounds['polygon'] = geom.wkt
        print(f"Bounds updated: {bounds}")

    # create the map
    m = Map(center=center, zoom=zoom, basemap=basemaps.Esri.WorldTopoMap)

    # add drawing tool for rectangles only
    draw_control = DrawControl(rectangle={'shapeOptions': {'color': '#fca45d'}})
    draw_control.polyline = {}
    draw_control.polygon = {}
    draw_control.circle = {}
    draw_control.circlemarker = {}
    draw_control.on_draw(_handle_draw)
    m.add_control(draw_control)

    # helper to fetch & layer GeoJSON
    def _add_layer(url, name, color):
        resp = requests.get(url)
        resp.raise_for_status()
        gj = GeoJSON(data=resp.json(), name=name, style={'color': color})
        m.add_layer(gj)

    # add the three catalog layers
    _add_layer(three_dep_url, "3DEP datasets", "#228B22")
    _add_layer(noaa_url,     "NOAA datasets", "#0000CD")
    _add_layer(ot_url,       "OpenTopography datasets", "#fca45d")

    # legend
    legend = LegendControl({
        "3DEP datasets": "#228B22",
        "NOAA datasets": "#0000CD",
        "OpenTopography datasets": "#fca45d"
    }, name="Legend", position="topright")
    m.add_control(legend)

    return m, bounds

def define_bounds_manual(south: float, north: float, west: float, east: float) -> dict:
    """
    Define an AOI by manually entering latitude/longitude bounds.
    
    Parameters
    ----------
    south : float
        Minimum latitude.
    north : float
        Maximum latitude.
    west : float
        Minimum longitude.
    east : float
        Maximum longitude.
    
    Returns
    -------
    dict
        A dictionary with keys:
          - 'south', 'north', 'west', 'east': the input bounds
          - 'polygon': WKT of the rectangle defined by those bounds
    """
    # create a rectangular polygon from the bounds
    poly = box(west, south, east, north)
    
    # extract the bounds
    bounds = {
        'south': south,
        'north': north,
        'west':  west,
        'east':  east,
        'polygon': poly.wkt
    }
    return bounds

def define_bounds_from_file(vector_path: str, target_crs: str = 'EPSG:4326') -> dict:
    """
    Define an AOI by uploading a vector file (shapefile or GeoJSON), detect its CRS,
    optionally reproject to a target CRS, and compute bounds.

    Parameters
    ----------
    vector_path : str
        Path to a polygon vector file. Supported formats:
        - Shapefile (.shp, with accompanying .shx/.dbf/etc.)
        - GeoJSON (.geojson or .json)
    target_crs : str, optional
        The CRS to which geometries should be reprojected for bounds calculation
        (default is 'EPSG:4326').

    Returns
    -------
    dict
        A dictionary with keys:
          - 'south', 'north', 'west', 'east': the bounding box of the file’s geometry
            in the target CRS
          - 'polygon': WKT of the full uploaded geometry (union of all features)
            in the target CRS
          - 'crs': the original CRS of the input file as a string
    """
    # Read the file into a GeoDataFrame
    gdf = gpd.read_file(vector_path)
    if gdf.empty:
        raise ValueError(f"No geometries found in {vector_path!r}")

    # Detect the original CRS
    original_crs = gdf.crs
    if original_crs is None:
        raise ValueError(f"CRS is undefined for {vector_path!r}")

    # Reproject to the target CRS if necessary
    if original_crs.to_string() != target_crs:
        gdf = gdf.to_crs(target_crs)

    # Merge all geometries into one
    geom_union = gdf.geometry.union_all()
    
    # Extract the bounding box
    minx_u, miny_u, maxx_u, maxy_u = geom_union.bounds
    
    # If the union is a MultiPolygon, replace it with its bounding‐box polygon
    if geom_union.geom_type == "MultiPolygon":
        geom_union = box(minx_u, miny_u, maxx_u, maxy_u)

    return {
        'south': miny_u,
        'north': maxy_u,
        'west': minx_u,
        'east': maxx_u,
        'polygon': geom_union.wkt,
        'crs': original_crs.to_string()
    }

def fetch_ot_catalog(
    base_url: str = "https://portal.opentopography.org/API",
    endpoint: str = "/otCatalog",
    bounds: dict = None,
    polygon: str = None,
    product_format: str = "PointCloud",
    detail: bool = False,
    output_format: str = "json",
    include_federated: bool = True,
    save_file: bool = True,
    filename: str = None,
    **request_kwargs
):
    """
    Query the OpenTopography API and optionally save the results to a file.

    Parameters
    ----------
    base_url : str
        The API base URL.
    endpoint : str
        The API endpoint (e.g. "/otCatalog").
    bounds : dict, optional
        A dict with keys 'west','south','east','north' specifying a WGS84 bbox.
        Required if `polygon` is not provided.
    polygon : str, optional
        A WKT polygon string. If given, bbox (`bounds`) will be ignored.
    product_format : str
        Data product format: "PointCloud" or "Raster".
    detail : bool
        If True, request detailed metadata.
    output_format : str
        "json" (default) or "xml".
    include_federated : bool
        Whether to include federated datasets.
    save_file : bool
        If True, write the response to disk; otherwise return parsed data.
    filename : str, optional
        Filename to save to. If None, defaults to "results.<output_format>".
    **request_kwargs
        Additional keyword args passed to requests.get (e.g., headers, timeout).

    Returns
    -------
    dict or bytes or None
        If save_file is False, returns the parsed JSON (when output_format="json")
        or raw bytes (when output_format!="json"). If save_file is True, returns None.
    """
    # Build URL and params
    url = base_url.rstrip("/") + endpoint
    params = {
        "productFormat": product_format,
        "detail": detail,
        "outputFormat": output_format,
        "include_federated": include_federated,
    }

    if polygon:
        params["polygon"] = polygon
    elif bounds:
        params.update({
            "minx": bounds["west"],
            "miny": bounds["south"],
            "maxx": bounds["east"],
            "maxy": bounds["north"],
        })
    else:
        raise ValueError("Either `bounds` or `polygon` must be provided.")

    # Perform the request
    response = requests.get(url, params=params, **request_kwargs)
    response.raise_for_status()

    # Handle output
    if save_file:
        if filename is None:
            filename = f"results.{output_format}"
        with open(filename, "wb") as f:
            f.write(response.content)
        print(f"Saved output to {filename}")
        return None
    else:
        if output_format.lower() == "json":
            return response.json()
        else:
            return response.content

In [43]:
def define_bounds_from_file(vector_path: str, target_crs: str = 'EPSG:4326') -> dict:
    """
    Define an AOI by uploading a vector file (shapefile or GeoJSON), detect its CRS,
    optionally reproject to a target CRS, and compute bounds.

    Parameters
    ----------
    vector_path : str
        Path to a polygon vector file. Supported formats:
        - Shapefile (.shp, with accompanying .shx/.dbf/etc.)
        - GeoJSON (.geojson or .json)
    target_crs : str, optional
        The CRS to which geometries should be reprojected for bounds calculation
        (default is 'EPSG:4326').

    Returns
    -------
    dict
        A dictionary with keys:
          - 'south', 'north', 'west', 'east': the bounding box of the file’s geometry
            in the target CRS
          - 'polygon': WKT of the full uploaded geometry (union of all features)
            in the target CRS
          - 'crs': the original CRS of the input file as a string
    """
    # Read the file into a GeoDataFrame
    gdf = gpd.read_file(vector_path)
    if gdf.empty:
        raise ValueError(f"No geometries found in {vector_path!r}")

    # Detect the original CRS
    original_crs = gdf.crs
    if original_crs is None:
        raise ValueError(f"CRS is undefined for {vector_path!r}")

    # Reproject to the target CRS if necessary
    if original_crs.to_string() != target_crs:
        gdf = gdf.to_crs(target_crs)

    # Merge all geometries into one
    geom_union = gdf.geometry.union_all()
    
    # Extract the bounding box
    minx_u, miny_u, maxx_u, maxy_u = geom_union.bounds
    
    # If the union is a MultiPolygon, replace it with its bounding‐box polygon
    if geom_union.geom_type == "MultiPolygon":
        geom_union = box(minx_u, miny_u, maxx_u, maxy_u)

    return {
        'south': miny_u,
        'north': maxy_u,
        'west': minx_u,
        'east': maxx_u,
        'polygon': geom_union.wkt,
        'crs': original_crs.to_string()
    }

<a id="#3-data-access-and-processing"></a>
## 3. Data Access and Processing

<a id="#31-define-area-of-interest"></a>
### 3.1. Define Area of Interest

To specify the geographic region for your dataset search, you have three options. You can 1) manually enter latitude and longitude bounds if you know the exact coordinates of your area of interest; 2) upload a shapefile or GEOJSON to automatically populate the same extent variables; 3) draw a bounding box directly on the interactive map embedded in this notebook: simply click and drag to sketch the rectangle around your target area, and the notebook captures both the corner coordinates and the equivalent WKT polygon for your API queries.

<a id="#311-option-1-draw-a-bounding-box-on-an-interactive-map"></a>
#### 3.1.1. Option 1: Draw a bounding box on an interactive map

In [11]:
m, global_bounds = init_ot_catalog_map()
display(m)
print ("Drawn bounds:", global_bounds)

Map(center=[39.8283, -98.5795], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

Drawn bounds: {'south': None, 'north': None, 'west': None, 'east': None, 'polygon': None}


<a id="#312-option-2-define-bounds-manually"></a>
#### 3.1.2. Option 2: Define bounds manually

In [24]:
global_bounds = define_bounds_manual(
    south=34.0,
    north=35.0,
    west=-118.5,
    east=-117.5
)
print("Manual bounds:", global_bounds)

Manual bounds: {'south': 34.0, 'north': 35.0, 'west': -118.5, 'east': -117.5, 'polygon': 'POLYGON ((-117.5 34, -117.5 35, -118.5 35, -118.5 34, -117.5 34))'}


<a id="#313-option-3-define-bounds-using-an-uploaded-shapefile"></a>
#### 3.1.3. Option 3: Define bounds using an uploaded file

In [47]:
vector_path = "test_shapefile.shp"  # Replace with your file path 
global_bounds = define_bounds_from_file(vector_path)
print("File‐derived bounds:", bounds)

File‐derived bounds: {'south': 39.812701501567226, 'north': 39.83787678884803, 'west': -85.67939879949874, 'east': -85.66006232462236, 'polygon': 'MULTIPOLYGON (((-85.66277575663145 39.82106676733368, -85.66305584356643 39.812701501567226, -85.67399998193764 39.81400961214544, -85.67138168483542 39.82134757286536, -85.66277575663145 39.82106676733368)), ((-85.66006232462236 39.83322432802182, -85.66654461406223 39.827477476275575, -85.67939879949874 39.833900886868406, -85.67708078211062 39.83787678884803, -85.66006232462236 39.83322432802182)))', 'crs': 'EPSG:32616'}


<a id="#32-use-ot-catalog-to-find-datasets"></a>
### 3.2. Use OT catalog to find datasets

In [27]:
data = fetch_ot_catalog(
    bounds = global_bounds,
    product_format =  "PointCloud",
    detail = True,
    output_format = "json",
    include_federated = True,
    save_file = True,
    )

Saved output to results.json


In [ ]:
data = fetch_ot_catalog(
    polygon = global_bounds['polygon'],
    product_format =  "PointCloud",
    detail = True,
    output_format = "json",
    include_federated = True,
    save_file = True,
    )

SyntaxError: invalid syntax (1216668390.py, line 1)

<a id="#4-conclusions"></a>
## 4. Conclusions

<a id="#5-resources"></a>
## 5. Resources

<a id="#6-funding-keywords-and-citation"></a>
## 6. Funding, Keywords, and Citation